In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv("Working_file_v13_1.csv")

In [ ]:
df

In [58]:
selected_df = df[["ye1_38b", "ye1_38c", "total_expend", "status", "hhtype", "y_nfm", "total_food"]] 

In [59]:
selected_df = selected_df[selected_df["total_expend"] != 0]

In [60]:
selected_df.shape

(4542, 7)

In [61]:
# Функция для вычисления перцентиля для каждого значения
#selected_df['percentile'] = selected_df['total_expend'].rank(pct=True) * 100

In [62]:
selected_df['percentile'] = pd.qcut(selected_df['total_expend'], q=100, labels=False) + 1

In [63]:
selected_df.head()

,ye1_38b,ye1_38c,total_expend,status,hhtype,y_nfm,total_food,percentile
0,0.0,0.00,31943.421875,3.0,глава Женщина пенс возраста,1.0,9099.440430,52
1,0.0,0.00,25445.679688,3.0,глава Женщина пенс возраста,1.0,6879.679688,41
2,1.0,334.29,10472.860352,3.0,глава Женщина пенс возраста,1.0,3852.860107,8
3,0.0,0.00,56543.460938,3.0,глава Мужчина трудоспос возраста,4.0,26568.109375,79
4,0.0,0.00,53498.511719,3.0,глава Мужчина трудоспос возраста,3.0,20237.902344,77


In [65]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

preprocessing = ColumnTransformer(
    transformers=[
        ("onehot", OneHotEncoder(handle_unknown='ignore'), ["hhtype"])  # Имя трансформера, трансформер, список столбцов
    ],
    remainder="passthrough"  # Оставить остальные столбцы без изменений
)


In [66]:
train = selected_df[(selected_df["ye1_38b"] != 0) & (selected_df["ye1_38c"] != 0)]

In [67]:
test = selected_df[(selected_df["ye1_38b"] == 0) | (selected_df["ye1_38c"] == 0)]

In [68]:
test.shape

(2635, 8)

In [69]:
train.shape

(1907, 8)

In [70]:
# Преобразование значений столбца 'y' в кортежи
train['y'] = train[['ye1_38b', 'ye1_38c']].apply(tuple, axis=1)

# Затем преобразование в массив numpy
y_train = np.array(train['y'].tolist())

X_train = train.drop(columns = ["y", "ye1_38b", "ye1_38c"])

/tmp/ipykernel_13893/3443332265.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['y'] = train[['ye1_38b', 'ye1_38c']].apply(tuple, axis=1)


In [71]:
X_test = test.drop(columns = ["ye1_38b", "ye1_38c"])

In [72]:
X_train

,total_expend,status,hhtype,y_nfm,total_food,percentile
2,10472.860352,3.0,глава Женщина пенс возраста,1.0,3852.860107,8
6,71090.804688,3.0,глава Мужчина трудоспос возраста,5.0,15743.719727,87
8,62335.191406,3.0,глава Мужчина трудоспос возраста,5.0,22405.312500,83
9,46287.742188,3.0,глава Мужчина трудоспос возраста,2.0,13868.550781,70
10,33488.238281,3.0,глава Мужчина трудоспос возраста,2.0,9511.508789,54
...,...,...,...,...,...,...
4537,15413.950195,1.0,глава Женщина пенс возраста,1.0,7033.950195,19
4538,33719.800781,3.0,глава Женщина трудоспос возраста,2.0,10450.730469,55
4539,39771.000000,3.0,глава Мужчина трудоспос возраста,2.0,19292.898438,63
4541,85950.710938,3.0,глава Мужчина трудоспос возраста,5.0,20987.541016,91


In [73]:
from sklearn.ensemble import RandomForestRegressor

model = make_pipeline(preprocessing, RandomForestRegressor(random_state=42))
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [74]:
X_test[['ye1_38b', 'ye1_38c']] = pd.DataFrame(y_pred, index=X_test.index)

In [75]:
final_train = train.drop(columns="y")

In [76]:
merged = pd.concat([final_train, X_test])

In [77]:
merged.sort_index(inplace=True)

In [78]:
print(selected_df.shape)
print(merged.shape)

(4542, 8)
(4542, 8)


In [79]:
merged["price_per_liter"] = merged["ye1_38c"] / merged["ye1_38b"]

In [80]:
merged["price_per_liter"].median()

np.float64(385.71222222222224)

In [81]:
merged.head()

,ye1_38b,ye1_38c,total_expend,status,hhtype,y_nfm,total_food,percentile,price_per_liter
0,1.160,452.3573,31943.421875,3.0,глава Женщина пенс возраста,1.0,9099.440430,52,389.963190
1,1.031,365.8707,25445.679688,3.0,глава Женщина пенс возраста,1.0,6879.679688,41,354.869738
2,1.000,334.2900,10472.860352,3.0,глава Женщина пенс возраста,1.0,3852.860107,8,334.290000
3,1.167,469.8428,56543.460938,3.0,глава Мужчина трудоспос возраста,4.0,26568.109375,79,402.607369
4,1.001,408.3949,53498.511719,3.0,глава Мужчина трудоспос возраста,3.0,20237.902344,77,407.986913


## Additional columns

In [82]:
df.head()

,YID_H,y_origsm,psu,status,site,FAMILYN,month,y_nfm,ye1_1b,ye1_1c,...,ye20_2d,ye20_3d,ye20_4d,ye20_5d,size,total_food,total_notfood,total_expend,SETT_TYP,hhtype
0,10003.0,1.0,6.0,3.0,1.0,3.0,10.0,1.0,0.0,0.00,...,0.000000,0.0,0.0,0.0,1.0,9099.440430,22843.982422,31943.421875,ПГТ,глава Женщина пенс возраста
1,10012.0,1.0,6.0,3.0,1.0,12.0,10.0,1.0,0.4,308.57,...,5975.304114,0.0,2000.0,0.0,1.0,6879.679688,18566.000000,25445.679688,ПГТ,глава Женщина пенс возраста
2,10014.0,1.0,6.0,3.0,1.0,14.0,10.0,1.0,0.4,175.71,...,0.000000,0.0,0.0,0.0,1.0,3852.860107,6620.000000,10472.860352,ПГТ,глава Женщина пенс возраста
3,10016.0,1.0,6.0,3.0,1.0,16.0,10.0,4.0,1.6,342.86,...,0.000000,1500.0,0.0,0.0,4.0,26568.109375,29975.351562,56543.460938,ПГТ,глава Мужчина трудоспос возраста
4,10035.0,1.0,6.0,3.0,1.0,35.0,10.0,3.0,0.4,162.86,...,0.000000,0.0,0.0,0.0,3.0,20237.902344,33260.609375,53498.511719,ПГТ,глава Мужчина трудоспос возраста


In [83]:
df["total_expend_per_person"] = df["total_expend"] / df["y_nfm"]

In [84]:
merged["total_expend_per_person"] = merged["total_expend"] / merged["y_nfm"]
merged["total_food_per_person"] = merged["total_food"] / merged["y_nfm"]
merged["ye1_38c_per_person"] = merged["ye1_38c"] / merged["y_nfm"]


In [85]:
merged.to_csv("merged.csv", index=False)

In [86]:
merged.shape

(4542, 12)